In [6]:
import os
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import sum,avg,max,count
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import col, unix_timestamp, from_unixtime
from functools import reduce


from pyspark.sql import functions as f
from pyspark.sql.functions import when
import matplotlib.pyplot as plt

from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import Query

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans

In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=5ae4eaa4ca4cf67cfc2c099eb7e629068e52d389217e9c3f0f0a8be15e787b9b
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [4]:
pip install dune_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.6/269.

In [7]:
import dunebuggy
from dunebuggy import Dune

dune = Dune()

In [5]:
pip install dunebuggy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.2 MB/s eta 0:00:00


In [8]:
conf = pyspark.SparkConf()

In [9]:
spark = SparkSession.builder.appName("Dune").getOrCreate()

In [10]:
def dataframe(a):
    df_new = a.df[['block_number', 'block_time', 'buyer', 'category', 'evt_type', 'nft_contract_address', 'nft_contract_addresses_array',
               'nft_project_name', 'nft_token_id', 'nft_token_ids_array', 'number_of_items', 'original_amount', 'original_currency',
               'platform', 'seller', 'trade_id', 'trade_type', 'usd_amount']]
    df_new['block_time'] =  pd.to_datetime(df_new['block_time'], infer_datetime_format=True)
    return df_new

In [1]:
query_list_a = [2201677, 2203423, 2203427,2203429,2203430,2203432,2203524,2203528,2203530, 
               2203531,2203532,2203534, 2203571, 2203573,2203575, 2203576, 2203577,2203580,
               2203581, 2203582, 2203608, 2203614, 2203616, 2203617, 2203618, 2203620, 2203622,
               2203623, 2203624, 2203625, 2203758, 2203761, 2203762, 2203767, 2203770, 2203771,
               2203772,2203774, 2203775, 2203777, 2282182, 2282187, 2282189, 2282192, 2282196, 
               2282216, 2282229, 2282239, 2282268, 2282324, 2282330, 2282825, 2282831, 2282845, 
               2282851, 2282865, 2282879, 2282885, 2282897, 2282913, 2282918, 2282925, 2282971, 2282990]

In [17]:
query_list_m = [2174066, 2167380, 2203067, 2203087, 2203123, 2203198, 2203199, 2203204,  2203242, 
         2203358, 2203364, 2203384, 2203358, 2203392, 2203415,  2203418, 2203419,
         2203421, 2203589, 2203621 ,2203637 , 2203644  , 2207729 , 2207754 ,
        2207790, 2207797  , 2207812  , 2207847 , 2207865 , 2207878, 2207905,
        2207914, 2207921, 2232809, 2232830, 2232838, 2232855, 2232872, 2232889,
        2232911, 2258257, 2258262, 2258274, 2258291, 2258322, 2258389, 2258402, 2258433, 2260264, 2260533,
        2260731, 2260783, 2260862, 2260902, 2261035, 2261073, 2261179, 2261823, 2261824, 2261825, 2261827,
        2261828, 2261829, 2261834]

In [ ]:
a = dune.fetch_query(2203199)
df1=dataframe(a)

<ipython-input-11-27f59a71c74b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['block_time'] =  pd.to_datetime(df_new['block_time'], infer_datetime_format=True)


In [ ]:
df1['number_of_items'] = df1['number_of_items'].fillna(0)

In [ ]:
df1['number_of_items'] = df1['number_of_items'].replace('<nil>', 0).astype(int)

In [ ]:
aa = dune.fetch_query(2167380)
df1=dataframe(a)

<ipython-input-11-27f59a71c74b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['block_time'] =  pd.to_datetime(df_new['block_time'], infer_datetime_format=True)


In [ ]:
df1 = df1.replace('<nil>', np.nan)

In [ ]:
df1.isna().sum()

block_number                        0
block_time                          0
buyer                               0
category                            0
evt_type                            0
nft_contract_address                0
nft_contract_addresses_array        0
nft_project_name                71208
nft_token_id                        0
nft_token_ids_array                 0
number_of_items                     0
original_amount                    74
original_currency                  74
platform                            0
seller                              0
trade_id                            0
trade_type                          0
usd_amount                        151
dtype: int64

In [ ]:
df1['block_time'] =  pd.to_datetime(df1['block_time'], infer_datetime_format=True)

In [ ]:
df1['original_amount'] = df1['original_amount'].replace('<nil>', 0.0).astype(float)
df1['usd_amount'] = df1['usd_amount'].replace('<nil>', 0.0).astype(float)
df1['number_of_items'] = df1['number_of_items'].replace('<nil>', 0).astype(int)
df1['nft_token_id'] = df1['nft_token_id'].astype(str)

In [ ]:
df_spark = spark.createDataFrame(data=df1)

In [ ]:
df_spark.dtypes

[('block_number', 'bigint'),
 ('block_time', 'timestamp'),
 ('buyer', 'string'),
 ('category', 'string'),
 ('evt_type', 'string'),
 ('nft_contract_address', 'string'),
 ('nft_contract_addresses_array', 'array<string>'),
 ('nft_project_name', 'string'),
 ('nft_token_id', 'string'),
 ('nft_token_ids_array', 'array<string>'),
 ('number_of_items', 'bigint'),
 ('original_amount', 'double'),
 ('original_currency', 'string'),
 ('platform', 'string'),
 ('seller', 'string'),
 ('trade_id', 'bigint'),
 ('trade_type', 'string'),
 ('usd_amount', 'double')]

In [ ]:
#df_spark.write.parquet('file:///C:/Users/MAITRI/Downloads/Parquet_New/output.parquet')

In [ ]:
# path = 'C:/Users/MAITRI/OneDrive/Documents/Parquet'
# if not os.path.exists(path):
#     os.mkdir(path)

# # write the DataFrame to Parquet file
# df_spark.write.parquet(path)

In [ ]:
# output_dir = 'file:///C:/Users/MAITRI/OneDrive/Documents/Parquet_N'

# # create the directory if it doesn't exist
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # write the DataFrame to Parquet file
# df_spark.write.parquet(output_dir)

In [14]:
import pyarrow.parquet as pq
import pyarrow as pa

In [15]:
# Convert the Pandas DataFrame to a PyArrow table
table = pa.Table.from_pandas(df1)

# Write the table to a Parquet file
pq.write_table(table, 'xyz.parquet')

NameError: ignored

In [20]:
from google.colab import files
#files.download('xyz.parquet')

In [18]:
for i in query_list_m:
  a = dune.fetch_query(i)
  df1=dataframe(a) 
  df1 = df1.replace('<nil>', np.nan)
  df1['block_time'] =  pd.to_datetime(df1['block_time'], infer_datetime_format=True)   
  df1['original_amount'] = df1['original_amount'].replace('<nil>', 0.0).astype(float)
  df1['usd_amount'] = df1['usd_amount'].replace('<nil>', 0.0).astype(float)
  df1['number_of_items'] = df1['number_of_items'].fillna(0)
  df1['number_of_items'] = df1['number_of_items'].replace('<nil>', 0).astype(int)
  df1['nft_token_id'] = df1['nft_token_id'].astype(str)
  table = pa.Table.from_pandas(df1)
  file_name=f"output_{i}.parquet"
  # Write the table to a Parquet file
  pq.write_table(table, file_name)

<ipython-input-10-27f59a71c74b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['block_time'] =  pd.to_datetime(df_new['block_time'], infer_datetime_format=True)
<ipython-input-10-27f59a71c74b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['block_time'] =  pd.to_datetime(df_new['block_time'], infer_datetime_format=True)
<ipython-input-10-27f59a71c74b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [21]:
for i in query_list_m:
  file_name=f"output_{i}.parquet"
  files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>